In [1]:
import time
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
import math
import sys
import os
sys.path.append("/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis")
from Manifold_Analysis import Manifold_analysis
sys.path.append("/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif")

#from algos.agents import A2CAgent
#from algos.models import ActorCnn, CriticCnn
#from algos.preprocessing.stack_frame import preprocess_frame, stack_frame

# importing required libraries
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA, IncrementalPCA
import pandas as pd
import seaborn as sns
import cv2
from tabulate import tabulate

from Generate_data_activity import Generate_data

import warnings
from scipy.sparse import (spdiags, SparseEfficiencyWarning, csc_matrix,
    csr_matrix, isspmatrix, dok_matrix, lil_matrix, bsr_matrix)
warnings.simplefilter('ignore',SparseEfficiencyWarning)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

/opt/anaconda3/envs/atari1.0/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


Device:  cpu


# A2C model

In [24]:
env = gym.make('SpaceInvaders-v0')


env.seed(0)

def stack_frames(frames, state, is_new=False):
    frame = preprocess_frame(state, (8, -12, -12, 4), 84)
    frames = stack_frame(frames, frame, is_new)

    return frames


INPUT_SHAPE = (4, 84, 84)
ACTION_SIZE = env.action_space.n
SEED = 0
GAMMA = 0.99           # discount factor
ALPHA= 0.0001          # Actor learning rate
BETA = 0.0005          # Critic learning rate
UPDATE_EVERY = 100     # how often to update the network 


epsilon_by_epsiode = lambda frame_idx: EPS_END + (EPS_START - EPS_END) * math.exp(-1. * frame_idx /EPS_DECAY)

start_epoch = 0
scores = []
scores_window = deque(maxlen=20)
length_trial = 500



agent_init = A2CAgent(INPUT_SHAPE, ACTION_SIZE, SEED, device, GAMMA, ALPHA, BETA, UPDATE_EVERY, ActorCnn, CriticCnn)
agent_1000 = A2CAgent(INPUT_SHAPE, ACTION_SIZE, SEED, device, GAMMA, ALPHA, BETA, UPDATE_EVERY, ActorCnn, CriticCnn)
agent_2000 = A2CAgent(INPUT_SHAPE, ACTION_SIZE, SEED, device, GAMMA, ALPHA, BETA, UPDATE_EVERY, ActorCnn, CriticCnn)
agent_3000 = A2CAgent(INPUT_SHAPE, ACTION_SIZE, SEED, device, GAMMA, ALPHA, BETA, UPDATE_EVERY, ActorCnn, CriticCnn)
agent_4000 = A2CAgent(INPUT_SHAPE, ACTION_SIZE, SEED, device, GAMMA, ALPHA, BETA, UPDATE_EVERY, ActorCnn, CriticCnn)
agent_5000 = A2CAgent(INPUT_SHAPE, ACTION_SIZE, SEED, device, GAMMA, ALPHA, BETA, UPDATE_EVERY, ActorCnn, CriticCnn)





Liste_agents = {
    "agent_a2c_init" : agent_init,
    "agent_a2c_1000" : agent_1000,
    "agent_a2c_2000" : agent_2000,
    "agent_a2c_3000" : agent_3000,
    "agent_a2c_4000" : agent_4000,
    "agent_a2c_5000" : agent_5000,
}




for a in Liste_agents.keys():
    if a != "agent_a2c_init":
        
        PATH = os.path.join("/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis","Models_training_a2c_space_invader",a+ ".pt")
        print(PATH)
        checkpoint = torch.load(PATH , map_location=torch.device('cpu'))
        Liste_agents[a].actor_net.load_state_dict(checkpoint['modelA_state_dict'])
        Liste_agents[a].critic_net.load_state_dict(checkpoint['modelB_state_dict'])
        Liste_agents[a].actor_optimizer.load_state_dict(checkpoint['optimizerA_state_dict'])
        Liste_agents[a].critic_optimizer.load_state_dict(checkpoint['optimizerB_state_dict'])
    
Names_hook = ["Conv_1","Conv_2","Conv_3","fc1"]
EPISODES = 100
Liste_activation = [[[[] for i in range(len(Names_hook))] for j in range(len( Liste_agents))] for episode in range(EPISODES)]
i = 0
Final_score = [[[] for j in range(len( Liste_agents))] for episode in range(EPISODES)]

agent_number = 0
for agent in Liste_agents.values():
    
    for episode in range(EPISODES):
        while i < length_trial:
            agent.registration()
            score = 0
            #print(Liste_activation[agent_number])
            
            Liste_activation[episode][agent_number] = [[] for i in range(len(Names_hook))]
            state = stack_frames(None, env.reset(), True)
            i = 0
            while True:
                #env.render()
                action, _, _ = agent.act(state)

                for h in range(len(Names_hook)):
                    Liste_activation[episode][agent_number][h].append(torch.flatten(agent.activation[Names_hook[h]]))  # If concatenate list directly with Liste_activation[agent_number][h] += torch.flatten(agent.activation[Names_hook[h]]) takes much more time

                next_state, reward, done, _ = env.step(action)
                score += reward
                state = stack_frames(state, next_state, False)
                i+=1
                if done:
                    print('\nEpisode :{} \tAgent number :{} \tFinal score: {:.2f} \tNumber of steps: {}'.format(episode,agent_number, score,i), end="")
                    
                    break 
            env.close()


            agent.detach()
        i = 0  
        Final_score[episode][agent_number].append(score)

    agent_number += 1


/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_a2c_space_invader/agent_a2c_1000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_a2c_space_invader/agent_a2c_2000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_a2c_space_invader/agent_a2c_3000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_a2c_space_invader/agent_a2c_4000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_a2c_space_invader/agent_a2c_5000.pt

Episode :0 	Agent number :0 	Final score: 225.00 	Number of steps: 918
Episode :1 	Agent number :0 	Final score: 205.00 	Number of steps: 763
Episode :2 	Agent number :0 	Final score: 155.00 	Nu

Episode :73 	Agent number :1 	Final score: 105.00 	Number of steps: 520
Episode :74 	Agent number :1 	Final score: 230.00 	Number of steps: 810
Episode :75 	Agent number :1 	Final score: 130.00 	Number of steps: 637
Episode :76 	Agent number :1 	Final score: 50.00 	Number of steps: 346
Episode :76 	Agent number :1 	Final score: 75.00 	Number of steps: 518
Episode :77 	Agent number :1 	Final score: 35.00 	Number of steps: 680
Episode :78 	Agent number :1 	Final score: 195.00 	Number of steps: 982
Episode :79 	Agent number :1 	Final score: 135.00 	Number of steps: 475
Episode :79 	Agent number :1 	Final score: 210.00 	Number of steps: 677
Episode :80 	Agent number :1 	Final score: 160.00 	Number of steps: 606
Episode :81 	Agent number :1 	Final score: 180.00 	Number of steps: 648
Episode :82 	Agent number :1 	Final score: 155.00 	Number of steps: 710
Episode :83 	Agent number :1 	Final score: 225.00 	Number of steps: 789
Episode :84 	Agent number :1 	Final score: 210.00 	Number of steps:

Episode :66 	Agent number :3 	Final score: 190.00 	Number of steps: 835
Episode :67 	Agent number :3 	Final score: 210.00 	Number of steps: 831
Episode :68 	Agent number :3 	Final score: 135.00 	Number of steps: 850
Episode :69 	Agent number :3 	Final score: 20.00 	Number of steps: 352
Episode :69 	Agent number :3 	Final score: 90.00 	Number of steps: 623
Episode :70 	Agent number :3 	Final score: 380.00 	Number of steps: 796
Episode :71 	Agent number :3 	Final score: 210.00 	Number of steps: 820
Episode :72 	Agent number :3 	Final score: 110.00 	Number of steps: 683
Episode :73 	Agent number :3 	Final score: 95.00 	Number of steps: 563
Episode :74 	Agent number :3 	Final score: 415.00 	Number of steps: 1603
Episode :75 	Agent number :3 	Final score: 110.00 	Number of steps: 511
Episode :76 	Agent number :3 	Final score: 135.00 	Number of steps: 645
Episode :77 	Agent number :3 	Final score: 105.00 	Number of steps: 707
Episode :78 	Agent number :3 	Final score: 160.00 	Number of steps

Episode :61 	Agent number :5 	Final score: 235.00 	Number of steps: 706
Episode :62 	Agent number :5 	Final score: 180.00 	Number of steps: 738
Episode :63 	Agent number :5 	Final score: 75.00 	Number of steps: 408
Episode :63 	Agent number :5 	Final score: 180.00 	Number of steps: 560
Episode :64 	Agent number :5 	Final score: 155.00 	Number of steps: 568
Episode :65 	Agent number :5 	Final score: 120.00 	Number of steps: 702
Episode :66 	Agent number :5 	Final score: 90.00 	Number of steps: 531
Episode :67 	Agent number :5 	Final score: 315.00 	Number of steps: 900
Episode :68 	Agent number :5 	Final score: 145.00 	Number of steps: 637
Episode :69 	Agent number :5 	Final score: 155.00 	Number of steps: 807
Episode :70 	Agent number :5 	Final score: 425.00 	Number of steps: 813
Episode :71 	Agent number :5 	Final score: 205.00 	Number of steps: 789
Episode :72 	Agent number :5 	Final score: 90.00 	Number of steps: 422
Episode :72 	Agent number :5 	Final score: 260.00 	Number of steps:

In [28]:
Generate_DATA = Generate_data(device)
Liste_activation,Final_score = Generate_DATA.run_episodes(structure_network = "a2c",length_trial = LENGTH_TRIAL,number_episode = EPISODES)

/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_a2c_space_invader/agent_a2c_1000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_a2c_space_invader/agent_a2c_2000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_a2c_space_invader/agent_a2c_3000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_a2c_space_invader/agent_a2c_4000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_a2c_space_invader/agent_a2c_5000.pt

Episode :0 	Agent number :0 	Final score: 110.00 	Number of steps: 599
Episode :1 	Agent number :0 	Final score: 160.00 	Number of steps: 644
Episode :2 	Agent number :0 	Final score: 10.00 	Num

KeyboardInterrupt: 

## Preprocessing

In [25]:
analysis = Manifold_analysis(length_trial)
Prepro_length = True
activity_layer = [[[[] for i in range(len(Names_hook))] for j in range(len( Liste_agents))] for episode in range(EPISODES)]

for agent_number in range(len(Liste_agents)):
    print("agent_number",agent_number)
    for layer in range(len(Names_hook)):
        for episode in range(EPISODES):
            activity_layer[episode][agent_number][layer] = analysis.prepro(Liste_activation[episode][agent_number][layer],Prepro_length)

agent_number 0
agent_number 1
agent_number 2
agent_number 3
agent_number 4
agent_number 5


## PCA

In [26]:
Names_hook = ["Conv_1","Conv_2","Conv_3","fc1"]


#fig = plt.figure(figsize=(20, 10))

L_threshold = [[[] for i in range(len(Names_hook))] for j in range(len( Liste_agents))] 


for agent_number in range(len(Liste_agents)):    
    for episode in range(EPISODES):
        for i in range(len(Names_hook)):

            X = activity_layer[episode][agent_number][i]
            covar_matrix = PCA(n_components = 50) #we have 20 features
            covar_matrix.fit(X)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios

            var=np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3)*100)
            #var #cumulative sum of variance explained with [n] features
            threshold = [i for i in range(len(var)) if var[i]>90]
            L_threshold[agent_number][i].append(threshold[0])


## Etude de l'influence de l'entrainement sur la dimension du manifold

In [27]:
table = [["Init"],["1000"],["2000"],["3000"],["4000"],["5000"]]

for agent_number in range(len(Liste_agents)):
    for i in range(len(Names_hook)):
        table[agent_number].append(np.mean(L_threshold[agent_number][i]))

        
print(tabulate(table, headers=["Number of Epoch","Conv_1","Conv_2","Conv_3","fc1"],tablefmt ="fancy_grid"))

╒═══════════════════╤══════════╤══════════╤══════════╤═══════╕
│ Number of Epoch   │   Conv_1 │   Conv_2 │   Conv_3 │   fc1 │
╞═══════════════════╪══════════╪══════════╪══════════╪═══════╡
│ Init              │    15.23 │    15.42 │    14.9  │ 14.91 │
├───────────────────┼──────────┼──────────┼──────────┼───────┤
│ 1000              │    19.28 │    16.48 │    10.04 │  0.65 │
├───────────────────┼──────────┼──────────┼──────────┼───────┤
│ 2000              │     8.85 │     6.52 │     4.59 │  0.24 │
├───────────────────┼──────────┼──────────┼──────────┼───────┤
│ 3000              │     7.45 │     4.72 │     2.72 │  2    │
├───────────────────┼──────────┼──────────┼──────────┼───────┤
│ 4000              │     7.84 │     4.62 │     3    │  1.63 │
├───────────────────┼──────────┼──────────┼──────────┼───────┤
│ 5000              │     7.57 │     5.96 │     4.16 │  1.4  │
╘═══════════════════╧══════════╧══════════╧══════════╧═══════╛


# DDQN Model

In [7]:
LENGTH_TRIAL = 500

EPISODES = 100

In [3]:
Generate_DATA = Generate_data(device)
Liste_activation,Final_score = Generate_DATA.run_episodes(structure_network = "ddqn",length_trial = LENGTH_TRIAL,number_episode = EPISODES)

/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_ddqn_space_invader/agent_ddqn_1000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_ddqn_space_invader/agent_ddqn_2000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_ddqn_space_invader/agent_ddqn_3000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_ddqn_space_invader/agent_ddqn_4000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_ddqn_space_invader/agent_ddqn_5000.pt


[W NNPACK.cpp:80] Could not initialize NNPACK! Reason: Unsupported hardware.



Episode :0 	Agent number :0 	Final score: 0.00 	Number of steps: 924
Episode :1 	Agent number :0 	Final score: 0.00 	Number of steps: 926
Episode :2 	Agent number :0 	Final score: 0.00 	Number of steps: 921
Episode :3 	Agent number :0 	Final score: 0.00 	Number of steps: 932
Episode :4 	Agent number :0 	Final score: 0.00 	Number of steps: 908
Episode :5 	Agent number :0 	Final score: 0.00 	Number of steps: 927
Episode :6 	Agent number :0 	Final score: 0.00 	Number of steps: 920
Episode :7 	Agent number :0 	Final score: 0.00 	Number of steps: 921
Episode :8 	Agent number :0 	Final score: 0.00 	Number of steps: 940
Episode :9 	Agent number :0 	Final score: 0.00 	Number of steps: 932
Episode :10 	Agent number :0 	Final score: 0.00 	Number of steps: 924
Episode :11 	Agent number :0 	Final score: 0.00 	Number of steps: 924
Episode :12 	Agent number :0 	Final score: 0.00 	Number of steps: 939
Episode :13 	Agent number :0 	Final score: 0.00 	Number of steps: 943
Episode :14 	Agent number :0 

Episode :29 	Agent number :2 	Final score: 285.00 	Number of steps: 962
Episode :30 	Agent number :2 	Final score: 285.00 	Number of steps: 954
Episode :31 	Agent number :2 	Final score: 285.00 	Number of steps: 973
Episode :32 	Agent number :2 	Final score: 285.00 	Number of steps: 976
Episode :33 	Agent number :2 	Final score: 285.00 	Number of steps: 986
Episode :34 	Agent number :2 	Final score: 285.00 	Number of steps: 970
Episode :35 	Agent number :2 	Final score: 285.00 	Number of steps: 982
Episode :36 	Agent number :2 	Final score: 285.00 	Number of steps: 961
Episode :37 	Agent number :2 	Final score: 285.00 	Number of steps: 979
Episode :38 	Agent number :2 	Final score: 285.00 	Number of steps: 982
Episode :39 	Agent number :2 	Final score: 285.00 	Number of steps: 963
Episode :40 	Agent number :2 	Final score: 285.00 	Number of steps: 967
Episode :41 	Agent number :2 	Final score: 285.00 	Number of steps: 975
Episode :42 	Agent number :2 	Final score: 285.00 	Number of ste

Episode :55 	Agent number :4 	Final score: 285.00 	Number of steps: 984
Episode :56 	Agent number :4 	Final score: 285.00 	Number of steps: 975
Episode :57 	Agent number :4 	Final score: 285.00 	Number of steps: 971
Episode :58 	Agent number :4 	Final score: 285.00 	Number of steps: 971
Episode :59 	Agent number :4 	Final score: 285.00 	Number of steps: 972
Episode :60 	Agent number :4 	Final score: 285.00 	Number of steps: 968
Episode :61 	Agent number :4 	Final score: 285.00 	Number of steps: 986
Episode :62 	Agent number :4 	Final score: 285.00 	Number of steps: 966
Episode :63 	Agent number :4 	Final score: 285.00 	Number of steps: 969
Episode :64 	Agent number :4 	Final score: 285.00 	Number of steps: 968
Episode :65 	Agent number :4 	Final score: 285.00 	Number of steps: 960
Episode :66 	Agent number :4 	Final score: 285.00 	Number of steps: 962
Episode :67 	Agent number :4 	Final score: 285.00 	Number of steps: 970
Episode :68 	Agent number :4 	Final score: 285.00 	Number of ste

In [10]:
Liste_agents = Generate_DATA.initialization_agents(structure_network ="ddqn")

/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_ddqn_space_invader/agent_ddqn_1000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_ddqn_space_invader/agent_ddqn_2000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_ddqn_space_invader/agent_ddqn_3000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_ddqn_space_invader/agent_ddqn_4000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_ddqn_space_invader/agent_ddqn_5000.pt


## Preprocessing and PCA

In [12]:
analysis = Manifold_analysis(LENGTH_TRIAL)
Names_hook = ["Conv_1","Conv_2","Conv_3","advantage","value "]


activity_layer = [[[[] for i in range(len(Names_hook))] for j in range(len( Liste_agents))] for episode in range(EPISODES)]

for agent_number in range(len(Liste_agents)):
    print("agent_number",agent_number)
    for layer in range(len(Names_hook)):
        for episode in range(EPISODES):
            activity_layer[episode][agent_number][layer] = analysis.prepro(Liste_activation[episode][agent_number][layer])

agent_number 0
agent_number 1
agent_number 2
agent_number 3
agent_number 4
agent_number 5


In [26]:

#fig = plt.figure(figsize=(20, 10))

L_threshold = [[[] for i in range(len(Names_hook))] for j in range(len( Liste_agents))] 


for agent_number in range(len(Liste_agents)):
    print("agent_number",agent_number)
    for episode in range(EPISODES):
        for i in range(len(Names_hook)-2):
            if i<3:
                COMPONENTS = 50
            elif i == 3:
                COMPONENTS = 5
            elif i == 4:
                COMPONENTS = 1
            X = activity_layer[episode][agent_number][i]
            covar_matrix = PCA(n_components = COMPONENTS) #we have 20 features
            covar_matrix.fit(X)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios

            var=np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3)*100)
            #var #cumulative sum of variance explained with [n] features
            threshold = [i for i in range(len(var)) if var[i]>60]
            L_threshold[agent_number][i].append(threshold[0])
            



agent_number 0
agent_number 1
agent_number 2
agent_number 3
agent_number 4
agent_number 5


## Influence of the training on the Manifold Dimension

In [27]:
table = [["Init"],["1000"],["2000"],["3000"],["4000"],["5000"]]

for agent_number in range(len(Liste_agents)):
    for i in range(len(Names_hook)):
        table[agent_number].append(np.mean(L_threshold[agent_number][i]))

        
print(tabulate(table, headers=["Number of Epoch","Conv_1","Conv_2","Conv_3","advantage","value "],tablefmt ="fancy_grid"))

╒═══════════════════╤══════════╤══════════╤══════════╤═════════════╤══════════╕
│ Number of Epoch   │   Conv_1 │   Conv_2 │   Conv_3 │   advantage │   value  │
╞═══════════════════╪══════════╪══════════╪══════════╪═════════════╪══════════╡
│ Init              │     3    │     3    │        3 │         nan │      nan │
├───────────────────┼──────────┼──────────┼──────────┼─────────────┼──────────┤
│ 1000              │     2.99 │     2    │        0 │         nan │      nan │
├───────────────────┼──────────┼──────────┼──────────┼─────────────┼──────────┤
│ 2000              │     3.11 │     0    │        0 │         nan │      nan │
├───────────────────┼──────────┼──────────┼──────────┼─────────────┼──────────┤
│ 3000              │     3    │     0    │        0 │         nan │      nan │
├───────────────────┼──────────┼──────────┼──────────┼─────────────┼──────────┤
│ 4000              │     2.43 │     1    │        0 │         nan │      nan │
├───────────────────┼──────────┼────────

/opt/anaconda3/envs/atari1.0/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda3/envs/atari1.0/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


# DQN

In [15]:
LENGTH_TRIAL = 500
EPISODES = 100

In [16]:
Generate_DATA = Generate_data(device)
Liste_activation,Final_score = Generate_DATA.run_episodes(structure_network = "dqn",length_trial = LENGTH_TRIAL,number_episode = EPISODES)

/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_dqn_space_invader/agent_dqn_1000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_dqn_space_invader/agent_dqn_2000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_dqn_space_invader/agent_dqn_3000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_dqn_space_invader/agent_dqn_4000.pt

Episode :0 	Agent number :0 	Final score: 270.00 	Number of steps: 717
Episode :1 	Agent number :0 	Final score: 270.00 	Number of steps: 723
Episode :2 	Agent number :0 	Final score: 270.00 	Number of steps: 712
Episode :3 	Agent number :0 	Final score: 270.00 	Number of steps: 719
Episode :4 	Agent number :0 	Final score: 270.00 	Number of steps: 723


Episode :16 	Agent number :2 	Final score: 105.00 	Number of steps: 394
Episode :16 	Agent number :2 	Final score: 210.00 	Number of steps: 1210
Episode :17 	Agent number :2 	Final score: 285.00 	Number of steps: 990
Episode :18 	Agent number :2 	Final score: 155.00 	Number of steps: 742
Episode :19 	Agent number :2 	Final score: 90.00 	Number of steps: 917
Episode :20 	Agent number :2 	Final score: 210.00 	Number of steps: 826
Episode :21 	Agent number :2 	Final score: 225.00 	Number of steps: 943
Episode :22 	Agent number :2 	Final score: 180.00 	Number of steps: 715
Episode :23 	Agent number :2 	Final score: 45.00 	Number of steps: 385
Episode :23 	Agent number :2 	Final score: 275.00 	Number of steps: 988
Episode :24 	Agent number :2 	Final score: 395.00 	Number of steps: 1672
Episode :25 	Agent number :2 	Final score: 180.00 	Number of steps: 706
Episode :26 	Agent number :2 	Final score: 210.00 	Number of steps: 714
Episode :27 	Agent number :2 	Final score: 80.00 	Number of step

Episode :64 	Agent number :3 	Final score: 0.00 	Number of steps: 382
Episode :64 	Agent number :3 	Final score: 0.00 	Number of steps: 387
Episode :64 	Agent number :3 	Final score: 0.00 	Number of steps: 657
Episode :65 	Agent number :3 	Final score: 0.00 	Number of steps: 374
Episode :65 	Agent number :3 	Final score: 0.00 	Number of steps: 663
Episode :66 	Agent number :3 	Final score: 0.00 	Number of steps: 383
Episode :66 	Agent number :3 	Final score: 0.00 	Number of steps: 386
Episode :66 	Agent number :3 	Final score: 0.00 	Number of steps: 654
Episode :67 	Agent number :3 	Final score: 0.00 	Number of steps: 385
Episode :67 	Agent number :3 	Final score: 0.00 	Number of steps: 386
Episode :67 	Agent number :3 	Final score: 0.00 	Number of steps: 660
Episode :68 	Agent number :3 	Final score: 0.00 	Number of steps: 380
Episode :68 	Agent number :3 	Final score: 0.00 	Number of steps: 387
Episode :68 	Agent number :3 	Final score: 0.00 	Number of steps: 661
Episode :69 	Agent n

In [17]:
Liste_agents = Generate_DATA.initialization_agents(structure_network ="dqn")

/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_dqn_space_invader/agent_dqn_1000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_dqn_space_invader/agent_dqn_2000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_dqn_space_invader/agent_dqn_3000.pt
/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_dqn_space_invader/agent_dqn_4000.pt


## Preprocessing and PCA

In [18]:
analysis = Manifold_analysis(LENGTH_TRIAL)
Names_hook = ["Conv_1","Conv_2","Conv_3","fc1"]

activity_layer = [[[[] for i in range(len(Names_hook))] for j in range(len( Liste_agents))] for episode in range(EPISODES)]

for agent_number in range(len(Liste_agents)):
    print("agent_number",agent_number)
    for layer in range(len(Names_hook)):
        for episode in range(EPISODES):
            activity_layer[episode][agent_number][layer] = analysis.prepro(Liste_activation[episode][agent_number][layer])

agent_number 0
agent_number 1
agent_number 2
agent_number 3
agent_number 4


In [19]:

#fig = plt.figure(figsize=(20, 10))

L_threshold = [[[] for i in range(len(Names_hook))] for j in range(len( Liste_agents))] 


for agent_number in range(len(Liste_agents)):
    print("agent_number",agent_number)
    for episode in range(EPISODES):
        print(episode)
        for i in range(len(Names_hook)):
            if i<3:
                COMPONENTS = 50
            elif i == 3:
                COMPONENTS = 5
            elif i == 4:
                COMPONENTS = 1
            X = activity_layer[episode][agent_number][i]
            covar_matrix = PCA(n_components = COMPONENTS) #we have 20 features
            covar_matrix.fit(X)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            if i >3:
                print(variance)
            var=np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3)*100)
            #var #cumulative sum of variance explained with [n] features
            threshold = [i for i in range(len(var)) if var[i]>90]
            L_threshold[agent_number][i].append(threshold[0])
            




agent_number 0
0


IndexError: list index out of range

## Influence of the training on the Manifold Dimension

In [ ]:
table = [["Init"],["1000"],["2000"],["3000"],["4000"]]

for agent_number in range(len(Liste_agents)):
    for i in range(len(Names_hook)):
        table[agent_number].append(np.mean(L_threshold[agent_number][i]))

        
print(tabulate(table, headers=["Number of Epoch","Conv_1","Conv_2","Conv_3","fc1 "],tablefmt ="fancy_grid"))